# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "../output_data/cities.csv"

# Read the mouse data and the study results
cities_data = pd.read_csv(cities_csv)
cities_data

,city_ID,city,cloudiness,country,date,humidity,latitude,longitude,max_temp,wind_speed
0,0,vaini,20.0,TO,1.586574e+09,65.0,-21.20,-175.20,80.60,8.05
1,1,new norfolk,75.0,AU,1.586574e+09,58.0,-42.78,147.06,57.00,17.22
2,2,dikson,91.0,RU,1.586574e+09,95.0,73.51,80.55,10.15,11.01
3,3,chokurdakh,39.0,RU,1.586574e+09,96.0,70.63,147.92,24.96,15.99
4,4,katsuura,20.0,JP,1.586574e+09,44.0,35.13,140.30,62.01,9.17
...,...,...,...,...,...,...,...,...,...,...
575,575,le port,40.0,RE,1.586574e+09,78.0,-20.94,55.29,77.00,11.41
576,576,yaring,20.0,TH,1.586574e+09,74.0,6.87,101.37,84.20,5.82
577,577,manyana,20.0,BW,1.586574e+09,56.0,-23.40,21.72,63.10,6.40
578,578,gimli,100.0,CA,1.586574e+09,75.0,50.63,-96.99,43.00,8.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#drop data w null values
complete_cities_data = cities_data.dropna()
#turn lat/lng into locations
locations = complete_cities_data[['latitude', 'longitude']].astype(float)
#convert humidity to float
humidity_rate = complete_cities_data['humidity'].astype(float)

In [5]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

#configure data for layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
#add layer
fig.add_layer(heat_layer)

#display
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
#narrowing results down to ideal weather
ideal_temp_low_df = complete_cities_data[complete_cities_data.max_temp >= 60]
ideal_temp_high_low_df = ideal_temp_low_df[ideal_temp_low_df.max_temp < 70]
ideal_wind_df = ideal_temp_high_low_df[ideal_temp_high_low_df.wind_speed < 7]
ideal_humidity = ideal_wind_df[ideal_wind_df.humidity < 75]
ideal_cloud_cover = ideal_humidity[ideal_humidity.cloudiness > 25]
ideal_cloud_cover

,city_ID,city,cloudiness,country,date,humidity,latitude,longitude,max_temp,wind_speed
30,30,ribeira grande,97.0,PT,1.586574e+09,73.0,38.52,-28.70,61.54,4.70
102,102,san ramon de la nueva oran,36.0,AR,1.586574e+09,60.0,-23.13,-64.33,62.62,2.21
153,153,san cristobal,69.0,VE,1.586574e+09,62.0,7.77,-72.22,69.01,2.08
156,156,lagoa,93.0,PT,1.586574e+09,72.0,39.05,-27.98,61.12,6.60
179,179,luangwa,50.0,ZM,1.586574e+09,73.0,-15.62,30.42,68.90,5.70
205,205,airai,58.0,TL,1.586574e+09,65.0,-8.93,125.41,67.87,1.12
252,252,nara,75.0,JP,1.586574e+09,47.0,34.69,135.80,60.01,4.70
265,265,san gabriel,75.0,US,1.586574e+09,54.0,34.10,-118.11,60.80,3.36
274,274,austin,75.0,US,1.586574e+09,64.0,30.27,-97.74,68.00,6.93
278,278,pacific grove,40.0,US,1.586574e+09,71.0,36.62,-121.92,60.80,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map